In [25]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px


df = pd.read_csv(r'../data/lokasjonsdata.csv')

app = Dash()

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign': 'center'}),
    dcc.Dropdown(
        options=[{'label': col, 'value': col} for col in df.columns[1:]],  # Columns 1–4
        value=df.columns[1],  # Default selection
        id='dropdown-selection'
    ),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(selected_column):
    # Use the first column as x-axis (dates) and the selected column as y-axis
    return px.line(df, x=df.columns[0], y=selected_column, title=f"{selected_column} Over Time")


In [ ]:
#skal brukes for å fremstille alt? hvis jeg tar meg tid til å gjøre det
app.run(jupyter_mode="inline",port=8050)


NameError: name 'app' is not defined